In [ ]:
import numpy as np

def forward_algorithm(observations, states, start_prob, trans_prob, emit_prob):
    n_states = len(states)
    T = len(observations)

    forward = np.zeros((n_states, T))

    # Initialization step
    for s in range(n_states):
        forward[s, 0] = start_prob[s] * emit_prob[s][observations[0]]

    # Recursion step
    for t in range(1, T):
        for s in range(n_states):
            forward[s, t] = sum(forward[prev_s, t-1] * trans_prob[prev_s][s] for prev_s in range(n_states)) * emit_prob[s][observations[t]]

    # Termination step
    prob = sum(forward[s, T-1] for s in range(n_states))
    return forward, prob

def backward_algorithm(observations, states, start_prob, trans_prob, emit_prob):
    n_states = len(states)
    T = len(observations)

    backward = np.zeros((n_states, T))

    # Initialization step
    for s in range(n_states):
        backward[s, T-1] = 1

    # Recursion step
    for t in range(T-2, -1, -1):
        for s in range(n_states):
            backward[s, t] = sum(trans_prob[s][next_s] * emit_prob[next_s][observations[t+1]] * backward[next_s, t+1] for next_s in range(n_states))

    # Termination step
    prob = sum(start_prob[s] * emit_prob[s][observations[0]] * backward[s, 0] for s in range(n_states))
    return backward, prob

def viterbi_algorithm(observations, states, start_prob, trans_prob, emit_prob):
    n_states = len(states)
    T = len(observations)

    viterbi = np.zeros((n_states, T))
    backpointer = np.zeros((n_states, T), dtype=int)

    # Initialization step
    for s in range(n_states):
        viterbi[s, 0] = start_prob[s] * emit_prob[s][observations[0]]
        backpointer[s, 0] = 0

    # Recursion step
    for t in range(1, T):
        for s in range(n_states):
            prob_transition = [viterbi[prev_s, t-1] * trans_prob[prev_s][s] for prev_s in range(n_states)]
            viterbi[s, t] = max(prob_transition) * emit_prob[s][observations[t]]
            backpointer[s, t] = np.argmax(prob_transition)

    # Termination step
    best_path_prob = max(viterbi[s, T-1] for s in range(n_states))
    best_path_pointer = np.argmax([viterbi[s, T-1] for s in range(n_states)])

    # Path backtracking
    best_path = [best_path_pointer]
    for t in range(T-1, 0, -1):
        best_path.insert(0, backpointer[best_path[0], t])

    return best_path, best_path_prob

# Example Usage
states = ["Sunny", "Rainy"]
observations = [0, 1, 1]  # Example: Encoded observations
start_prob = [0.6, 0.4]
trans_prob = [[0.7, 0.3], [0.4, 0.6]]
emit_prob = [[0.5, 0.5], [0.1, 0.9]]

# Run algorithms
fwd, prob_fwd = forward_algorithm(observations, states, start_prob, trans_prob, emit_prob)
bwd, prob_bwd = backward_algorithm(observations, states, start_prob, trans_prob, emit_prob)
best_path, prob_vit = viterbi_algorithm(observations, states, start_prob, trans_prob, emit_prob)

print("Forward Probabilities:")
print(fwd)
print("Probability of the observation sequence (Forward):", prob_fwd)

print("\nBackward Probabilities:")
print(bwd)
print("Probability of the observation sequence (Backward):", prob_bwd)

print("\nViterbi Best Path:", best_path)
print("Viterbi Best Path Probability:", prob_vit)


Forward Probabilities:
[[0.3      0.113    0.06007 ]
 [0.04     0.1026   0.085914]]
Probability of the observation sequence (Forward): 0.145984

Backward Probabilities:
[[0.4168 0.62   1.    ]
 [0.5236 0.74   1.    ]]
Probability of the observation sequence (Backward): 0.145984

Viterbi Best Path: [0, 1, 1]
Viterbi Best Path Probability: 0.04374
